In [1]:
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import json
import time
import random
from google.auth.transport.requests import Request
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.poolmanager import PoolManager
import requests

# Scopes cần thiết để truy cập YouTube Data API
scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

# Tên thư mục chứa video
VIDEO_FOLDER = 'C:\\Users\\nguyenmanhcuong1991\\Desktop\\video\\'

# Danh sách proxy
PROXIES = [
    "http://proxy1.example.com:8080",
    "http://proxy2.example.com:8080",
    "http://proxy3.example.com:8080"
]

# Tạo hàm để xác thực và ủy quyền với proxy
def get_authenticated_service(proxy=None):
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    session = requests.Session()
    if proxy:
        session.proxies = {
            "http": proxy,
            "https": proxy,
        }

    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        "client_secret.json", scopes)
    credentials = flow.run_local_server(port=8080, access_type='offline', prompt='consent', session=session)

    if proxy:
        # Create a transport with the specified proxy
        transport = Request(session=session)
        credentials.refresh(transport)

    return googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

# Hàm upload video lên YouTube
def upload_video(youtube, file, title, description, category, tags, privacyStatus):
    body = {
        "snippet": {
            "title": title,
            "description": description,
            "tags": tags,
            "categoryId": category
        },
        "status": {
            "privacyStatus": privacyStatus
        }
    }

    insert_request = youtube.videos().insert(
        part="snippet,status",
        body=body,
        media_body=file
    )

    response = insert_request.execute()
    return response["id"]

# Hàm tạo playlist
def create_playlist(youtube, title, description, privacyStatus):
    body = {
        "snippet": {
            "title": title,
            "description": description
        },
        "status": {
            "privacyStatus": privacyStatus
        }
    }

    playlists_insert_request = youtube.playlists().insert(
        part="snippet,status",
        body=body
    )

    response = playlists_insert_request.execute()
    return response["id"]

# Hàm thêm video vào playlist
def add_video_to_playlist(youtube, video_id, playlist_id):
    add_video_request = youtube.playlistItems().insert(
        part="snippet",
        body={
            "snippet": {
                "playlistId": playlist_id,
                "resourceId": {
                    "kind": "youtube#video",
                    "videoId": video_id
                }
            }
        }
    )

    add_video_request.execute()

def main():
    try:
        # Kiểm tra danh sách video có sẵn trên YouTube và lưu trữ ID vào một tập hợp
        existing_videos = set()
        search_request = youtube.search().list(
            part="id",
            forMine=True,
            type="video",
            maxResults=50  # Số lượng video tối đa trong mỗi lần gọi
        )
        while search_request:
            try:
                search_response = search_request.execute()
                for item in search_response.get("items", []):
                    existing_videos.add(item["id"]["videoId"])  # Corrected to add only videoId
                search_request = youtube.search().list_next(search_request, search_response)
            except googleapiclient.errors.HttpError as e:
                if e.resp.status in [403] and 'quotaExceeded' in e.content.decode():
                    print('Quota exceeded, waiting for quota reset.')
                    time.sleep(600)  # Chờ 10 phút trước khi thử lại
                else:
                    raise e

        # Kiểm tra danh sách playlist có sẵn trên YouTube và lưu trữ ID vào một từ điển
        existing_playlists = {}
        playlist_request = youtube.playlists().list(
            part="snippet",
            mine=True,
            maxResults=50  # Số lượng playlist tối đa trong mỗi lần gọi
        )
        while playlist_request:
            try:
                playlist_response = playlist_request.execute()
                for playlist in playlist_response.get("items", []):
                    existing_playlists[playlist["snippet"]["title"]] = playlist["id"]
                playlist_request = youtube.playlists().list_next(playlist_request, playlist_response)
            except googleapiclient.errors.HttpError as e:
                if e.resp.status in [403] and 'quotaExceeded' in e.content.decode():
                    print('Quota exceeded, waiting for quota reset.')
                    time.sleep(600)  # Chờ 10 phút trước khi thử lại
                else:
                    raise e

        # Lặp qua danh sách tệp video trong thư mục
        for file in os.listdir(VIDEO_FOLDER):
            if file.endswith(".mp4"):
                parts = file.split('-')
                lop = parts[0]
                video_number = parts[1].split('.')[0]

                title = f"{lop}-{video_number}"
                description = f"Description for {title}"
                category = "22"  # Category ID cho Education
                tags = ["tag1", "tag2"]
                privacyStatus = "public"

                # Chọn ngẫu nhiên proxy từ danh sách
                proxy = random.choice(PROXIES)
                youtube = get_authenticated_service(proxy)

                try:
                    if title not in existing_videos:
                        # Nếu video chưa tồn tại trên YouTube, thực hiện tải lên
                        video_id = upload_video(youtube, os.path.join(VIDEO_FOLDER, file), title, description, category, tags, privacyStatus)
                        print(f"Uploaded video '{title}' with ID: {video_id}")

                        # Tạo playlist nếu playlist chưa tồn tại
                        if lop not in existing_playlists:
                            playlist_id = create_playlist(youtube, lop, f"Playlist for {lop}", "public")
                            existing_playlists[lop] = playlist_id
                            print(f"Created playlist '{lop}' with ID: {playlist_id}")

                        # Thêm video vào playlist
                        add_video_to_playlist(youtube, video_id, existing_playlists[lop])
                        print(f"Added video '{title}' to playlist '{lop}' with ID: {existing_playlists[lop]}")

                        # Thêm ID video mới vào tập hợp các video đã tồn tại
                        existing_videos.add(video_id)
                    else:
                        print(f"Video '{title}' already exists on YouTube")

                    # Thời gian nghỉ giữa các lần gọi API để tránh bị giới hạn
                    time.sleep(60)
                except googleapiclient.errors.HttpError as e:
                    if e.resp.status in [403] and 'quotaExceeded' in e.content.decode():
                        print('Quota exceeded, waiting for quota reset.')
                        time.sleep(600)  # Chờ 10 phút trước khi thử lại
                    else:
                        raise e

    except Exception as e:
        print("An error occurred:", e)

if __name__ == "__main__":
    main()


An error occurred: local variable 'youtube' referenced before assignment
